Reference: https://github.com/toushalipal6991/400-Birds-Species-Classification/tree/main/Prod-Deployment

In [4]:
import tensorflow as tf
import numpy as np
import keras
import os
from PIL import Image
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_prep
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import warnings
warnings.filterwarnings('ignore')
import pickle
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok

In [2]:
cd /content/drive/MyDrive/Colab Notebooks/deploy

/content/drive/MyDrive/Colab Notebooks/deploy


In [6]:

app = Flask(__name__)
run_with_ngrok(app)

a_file = open("/content/drive/MyDrive/Colab Notebooks/deploy/data.pkl", "rb")
labels = pickle.load(a_file)

# loading our pretrained model

model=keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/deploy/bestvgg19.h5')



def predict_species(path):
  image = tf.keras.utils.load_img(path,target_size=(224,224)) 
  img = vgg19_prep(img_to_array(image))
  img = np.expand_dims(img, axis=0)

  prediction = model.predict(img)
  sorted_preds = np.sort(prediction)[0][::-1]
  top = np.argsort(prediction)[0][::-1][0]
  return 'This Bird Belongs To:'+ labels.get(top)



@app.route("/")
def home():
	return render_template('index.html')
 

@app.route('/predict',methods=['POST'])
def predict():
    if request.method == "POST":
        imagefile=request.files['imagefile']
        if imagefile:
            image_path = "./static/" + imagefile.filename
            imagefile.save(image_path)
            return render_template('index.html',prediction=predict_species(image_path),imageloc=imagefile.filename)
    return render_template('index.html',prediction=predict_species(image_path),imageloc = None)
app.run()


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ee2c-34-67-174-162.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Jun/2022 20:03:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 20:03:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Jun/2022 20:03:32] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 20:03:32] "GET /static/1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 20:03:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 20:03:44] "GET /static/7.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 20:03:55] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 20:03:56] "GET /static/14.jpg HTTP/1.1" 200 -
